In [3]:
def create_multimodal_request(question_data, case_details, case_id, question_id, agent, thread):
    # Parse required figures
    try:
        # Try multiple ways of parsing figures
        if isinstance(question_data["figures"], str):
            try:
                required_figures = json.loads(question_data["figures"])
            except json.JSONDecodeError:
                required_figures = [question_data["figures"]]
        elif isinstance(question_data["figures"], list):
            required_figures = question_data["figures"]
        else:
            required_figures = [str(question_data["figures"])]
    except Exception as e:
        print(f"Error parsing figures: {e}")
        required_figures = []

    # Ensure each figure starts with "Figure "
    required_figures = [
        fig if fig.startswith("Figure ") else f"Figure {fig}" for fig in required_figures
    ]

    subfigures = []
    for figure in required_figures:
        # Handle both regular figures and those with letter suffixes
        base_figure_num = "".join(filter(str.isdigit, figure))
        figure_letter = "".join(filter(str.isalpha, figure.split()[-1])) or None

        # Find matching figures in case details
        matching_figures = [
            case_figure
            for case_figure in case_details.get("figures", [])
            if case_figure["number"] == f"Figure {base_figure_num}"
        ]

        if not matching_figures:
            print(f"No matching figure found for {figure} in case {case_id}")
            continue

        for case_figure in matching_figures:
            # If a specific letter is specified, filter subfigures
            if figure_letter:
                matching_subfigures = [
                    subfig
                    for subfig in case_figure.get("subfigures", [])
                    if subfig.get("number", "").lower().endswith(figure_letter.lower())
                    or subfig.get("label", "").lower() == figure_letter.lower()
                ]
                subfigures.extend(matching_subfigures)
            else:
                # If no letter specified, add all subfigures
                subfigures.extend(case_figure.get("subfigures", []))

    # Add images to content
    figure_prompt = ""
    image_urls = []

    for subfig in subfigures:
        if "number" in subfig:
            subfig_number = subfig["number"].lower().strip().replace(" ", "_") + ".jpg"
            subfig_path = os.path.join(FIGURES_DIR, case_id, subfig_number)
            figure_prompt += f"{subfig_number} located at {subfig_path}\n"
        if "url" in subfig:
            image_urls.append(subfig["url"])
        else:
            print(f"Subfigure missing URL: {subfig}")

    prompt = (
        f"Answer this question correctly using chain of thought reasoning and "
        "carefully evaluating choices. Solve using our own vision and reasoning and then"
        "use tools to complement your reasoning. Trust your own judgement over any tools.\n"
        f"{question_data['question']}\n{figure_prompt}"
    )

    try:
        start_time = time.time()

        final_response, agent_state = run_medrax(
            agent=agent, thread=thread, prompt=prompt, image_urls=image_urls
        )
        model_answer, agent_state = run_medrax(
            agent=agent,
            thread=thread,
            prompt="If you had to choose the best option, only respond with the letter of choice (only one of A, B, C, D, E, F)",
        )
        duration = time.time() - start_time

        log_entry = {
            "case_id": case_id,
            "question_id": question_id,
            "timestamp": datetime.now().isoformat(),
            "model": model_name,
            "temperature": temperature,
            "duration": round(duration, 2),
            "usage": "",
            "cost": 0,
            "raw_response": final_response,
            "model_answer": model_answer.strip(),
            "correct_answer": question_data["answer"][0],
            "input": {
                "messages": prompt,
                "question_data": {
                    "question": question_data["question"],
                    "explanation": question_data["explanation"],
                    "metadata": question_data.get("metadata", {}),
                    "figures": question_data["figures"],
                },
                "image_urls": [subfig["url"] for subfig in subfigures if "url" in subfig],
                "image_captions": [subfig.get("caption", "") for subfig in subfigures],
            },
            "agent_state": agent_state,
        }
        logging.info(json.dumps(log_entry))
        return final_response, model_answer.strip()

    except Exception as e:
        log_entry = {
            "case_id": case_id,
            "question_id": question_id,
            "timestamp": datetime.now().isoformat(),
            "model": model_name,
            "temperature": temperature,
            "status": "error",
            "error": str(e),
            "cost": 0,
            "input": {
                "messages": prompt,
                "question_data": {
                    "question": question_data["question"],
                    "explanation": question_data["explanation"],
                    "metadata": question_data.get("metadata", {}),
                    "figures": question_data["figures"],
                },
                "image_urls": [subfig["url"] for subfig in subfigures if "url" in subfig],
                "image_captions": [subfig.get("caption", "") for subfig in subfigures],
            },
        }
        logging.info(json.dumps(log_entry))
        print(f"Error processing case {case_id}, question {question_id}: {str(e)}")
        return "", ""


def load_benchmark_questions(case_id):
    benchmark_dir = "../benchmark/questions"
    return glob.glob(f"{benchmark_dir}/{case_id}/{case_id}_*.json")


def count_total_questions():
    total_cases = len(glob.glob("../benchmark/questions/*"))
    total_questions = sum(
        len(glob.glob(f"../benchmark/questions/{case_id}/*.json"))
        for case_id in os.listdir("../benchmark/questions")
    )
    return total_cases, total_questions


def main(tools):
    with open("../data/eurorad_metadata.json", "r") as file:
        data = json.load(file)

    total_cases, total_questions = count_total_questions()
    cases_processed = 0
    questions_processed = 0
    skipped_questions = 0

    print(f"Beginning benchmark evaluation for model {model_name} with temperature {temperature}\n")

    for case_id, case_details in data.items():
        if int(case_details["case_id"]) <= 17158:
            continue

        print(f"----------------------------------------------------------------")
        agent, thread = get_agent(tools)

        question_files = load_benchmark_questions(case_id)
        if not question_files:
            continue

        cases_processed += 1
        for question_file in question_files:
            with open(question_file, "r") as file:
                question_data = json.load(file)
                question_id = os.path.basename(question_file).split(".")[0]

            # agent, thread = get_agent(tools)
            questions_processed += 1
            final_response, model_answer = create_multimodal_request(
                question_data, case_details, case_id, question_id, agent, thread
            )

            # Handle cases where response is None
            if final_response is None:
                skipped_questions += 1
                print(f"Skipped question: Case ID {case_id}, Question ID {question_id}")
                continue

            print(
                f"Progress: Case {cases_processed}/{total_cases}, Question {questions_processed}/{total_questions}"
            )
            print(f"Case ID: {case_id}")
            print(f"Question ID: {question_id}")
            print(f"Final Response: {final_response}")
            print(f"Model Answer: {model_answer}")
            print(f"Correct Answer: {question_data['answer']}")
            print(f"----------------------------------------------------------------\n")

    print(f"\nBenchmark Summary:")
    print(f"Total Cases Processed: {cases_processed}")
    print(f"Total Questions Processed: {questions_processed}")
    print(f"Total Questions Skipped: {skipped_questions}")